One strategy of understanding is a hierarchachally nested series of models, at different levels of abstraction for which the principals at a higher level can be derived from those at a lower level. For example chemistry can be derived from the principles of physics.

Work of Pasupathy and Conor found that V4 cell's responses are well described by a receptive field over the angular position of curvature of object boundaries. While this is not an image computable model, it only has 4 parameters, is interpretable, and captures up to 60% of the variance of V4 cells. The image computable models Cadieu Model, HMAX, and SRF have all attempted to fit these data and have either failed to capture shape selectivity, or translation invariance of 
An image computable neural circuit model of units at a high level in the ventral stream 
A goal of visual neuroscience is to develop image computable models of units in the visual stream. Units further along in the ventral stream for a long time have seemed out of reach. Recently CNN's trained on object recongition tasks have been able to account for roughly half the variance of units in IT and V4. Unfortunately these models are not easily interpretable in part because the numbers of parameters is very high and so what about these responses they explain is unknown it could be trivial, or important properties.

Response Distribution

Sparsity 

Translation Invariance





